In [10]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Myth of programming

Consider the following code snippets, which side is faster?

In [ ]:
compare([do_render_code("flip_array.c", show="regswap"),do_render_code("flip_array.c", show="xorswap")])

In [4]:
render_code("flip_array.c")

// flip_array.c:1-41 (41 lines)
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void (*swap)(int* a, int* b);
void regswap(int* a, int* b)
{
    int temp = *a;
    *a = *b;
    *b = temp;
}

void xorswap(int* a, int* b)
{
    *a^=*b^=*a^=*b;
}


int main(int argc, char **argv)
{
    unsigned array_size = 131072;
    int *data;
    struct timeval time_start, time_end;
    array_size = atoi(argv[1]);
    if(argc > 2 && atoi(argv[2])==1)
        swap = xorswap;
    else
        swap = regswap;
    data = (int *)malloc(sizeof(int)*array_size);
    for (unsigned i = 0; i < array_size; ++i)
        data[i] = rand();
   gettimeofday(&time_start, NULL);
   for (unsigned i = 0; i < array_size; i++)
   {
        swap(&data[i],&data[array_size-1-i]);
   }
   gettimeofday(&time_end, NULL);
   fprintf(stderr, "data[array_size/2] = %d\t", data[rand()%131072]);
   fprintf(stderr, "flipped %lf seconds\n",((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
   return 0;
}

In [5]:
! make clean; make flip_array
! echo "Version A"
! for i in {1..10} ; do ./flip_array 2097152 0; done
! echo "Version B"
! for i in {1..10} ; do ./flip_array 2097152 1; done

rm -f arithmetics arithmetics_A arithmetics_B linked_list flip_array swap linked_list.s
cc -O3 -DHAVE_LINUX_PERF_EVENT_H flip_array.c -o flip_array
cc -S -O2 -DHAVE_LINUX_PERF_EVENT_H flip_array.c -o flip_array.s
Version A
data[array_size/2] = 507321095	flipped 0.001747 seconds
data[array_size/2] = 507321095	flipped 0.001711 seconds
data[array_size/2] = 507321095	flipped 0.001666 seconds
data[array_size/2] = 507321095	flipped 0.001709 seconds
data[array_size/2] = 507321095	flipped 0.001635 seconds
data[array_size/2] = 507321095	flipped 0.001642 seconds
data[array_size/2] = 507321095	flipped 0.001633 seconds
data[array_size/2] = 507321095	flipped 0.001649 seconds
data[array_size/2] = 507321095	flipped 0.001641 seconds
data[array_size/2] = 507321095	flipped 0.001637 seconds
Version B
data[array_size/2] = 507321095	flipped 0.001828 seconds
data[array_size/2] = 507321095	flipped 0.001802 seconds
data[array_size/2] = 507321095	flipped 0.001720 seconds
data[array_size/2] = 507321095	flipped 